In [2]:
import os
import json

from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# .env 파일 로드 (프로젝트 루트에 .env가 있다고 가정)
load_dotenv("api.env")

# Hugging Face API 토큰은 환경변수 HUGGINGFACE_HUB_TOKEN에 설정되어 있어야 합니다.
if not os.getenv("HUGGINGFACE_HUB_TOKEN"):
    raise ValueError("HUGGINGFACE_HUB_TOKEN이 .env 파일에 설정되어 있지 않습니다.")

# None 또는 빈 문자열이면 "정보 없음"으로 반환하는 함수
def clean_metadata(value):
    return value if value not in [None, ""] else "정보 없음"

# JSON 파일의 실제 키에 맞춰 FAQ 스타일 텍스트 생성 함수
def generate_selected_text(obj):
    return (
        f"Q: 서비스 이름은 무엇인가요?\nA: {clean_metadata(obj.get('서비스명'))}\n\n"
        f"Q: 서비스 ID는 무엇인가요?\nA: {clean_metadata(obj.get('서비스ID'))}\n\n"
        f"Q: 제공 부서는 어디인가요?\nA: {clean_metadata(obj.get('부서명'))}\n\n"
        f"Q: 서비스 분야는 무엇인가요?\nA: {clean_metadata(obj.get('서비스분야'))}\n\n"
        f"Q: 서비스 목적은 무엇인가요?\nA: {clean_metadata(obj.get('서비스목적요약'))}\n\n"
        f"Q: 지원 내용은 무엇인가요?\nA: {clean_metadata(obj.get('지원내용'))}\n\n"
        f"Q: 선정 기준은 무엇인가요?\nA: {clean_metadata(obj.get('선정기준'))}\n\n"
        f"Q: 신청 기한은 언제까지인가요?\nA: {clean_metadata(obj.get('신청기한'))}\n\n"
        f"Q: 신청 방법은 무엇인가요?\nA: {clean_metadata(obj.get('신청방법'))}\n\n"
        f"Q: 접수 기관은 어디인가요?\nA: {clean_metadata(obj.get('접수기관'))}\n\n"
    )

# JSON 파일 로드 (파일 경로를 실제 파일 위치에 맞게 수정)
json_file_path = r"./20250304.json"
with open(json_file_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)

if not isinstance(json_data, list):
    raise ValueError("JSON 데이터가 리스트 형태가 아닙니다.")

# 각 JSON 객체를 Document 객체로 변환 (메타데이터에도 실제 JSON 키 사용)
documents = []
for obj in json_data:
    text = generate_selected_text(obj)
    metadata = {
        "서비스ID": clean_metadata(obj.get("서비스ID")),
        "서비스명": clean_metadata(obj.get("서비스명")),
        "서비스목적요약": clean_metadata(obj.get("서비스목적요약")),
        "신청기한": clean_metadata(obj.get("신청기한")),
        "지원내용": clean_metadata(obj.get("지원내용")),
        "서비스분야": clean_metadata(obj.get("서비스분야")),
        "선정기준": clean_metadata(obj.get("선정기준")),
        "신청방법": clean_metadata(obj.get("신청방법")),
        "부서명": clean_metadata(obj.get("부서명")),
        "접수기관": clean_metadata(obj.get("접수기관"))
    }
    documents.append(Document(page_content=text, metadata=metadata))

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},  # 'cuda' 대신 'cpu' 사용
    encode_kwargs={'normalize_embeddings': True}
)


# FAISS 벡터스토어 생성 (전체 문서를 한 번에 인덱싱)
vectorstore = FAISS.from_documents(documents, embedding_model)

# FAISS 인덱스를 로컬에 저장할 디렉토리 지정 (없으면 생성)
persist_directory = r"./final"
os.makedirs(persist_directory, exist_ok=True)
vectorstore.save_local(persist_directory)
print(f"총 {len(documents)}개의 문서가 FAISS 벡터DB에 저장됨.")
print(f"저장 경로: {persist_directory}")

# 쿼리 테스트: 예시 쿼리 "귀어 창업 지원 대상은 누구인가요?"를 사용해 유사 문서 검색
query = "부산광역시 해운대구에 사는 28세 여자에게 맞는 서비스를 찾아줘. 생활안전에 도움이 될 현금 정책"
retrieved_docs = vectorstore.similarity_search(query, k=10)

print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n문서 {i}:")
    print(doc.page_content)
    print("메타데이터:", doc.metadata)

KeyboardInterrupt: 

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.load import dumps, loads
from operator import itemgetter

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# api.env 파일에서 환경 변수 로드
load_dotenv("api.env")

# 환경 변수에서 OPENAI_API_KEY 가져오기
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY가 api.env 파일에 설정되어 있지 않습니다.")

# ChatOpenAI 인스턴스 생성 시 api_key 전달
llm = ChatOpenAI(temperature=0, api_key=api_key)


# --- 1. 쿼리 생성 ---
# 입력 질문에 대해 여러 검색 쿼리를 생성하는 프롬프트 정의
template = (
    "You are a helpful assistant that generates multiple search queries based on a single input query.\n"
    "Generate multiple search queries related to: {question}\n"
    "Output (4 queries):"
)
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# 쿼리 생성 체인 구성: 프롬프트 -> LLM -> 문자열 파싱 -> 줄바꿈으로 분할
generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

# --- 2. 검색 결과 융합 ---
# Reciprocal Rank Fusion 함수: 여러 리스트의 검색 결과를 하나로 융합
def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    # 여기서는 최종적으로 문서 객체만 추출합니다.
    return [doc for doc, score in reranked_results]

# retriever는 이미 정의된 벡터스토어 리트리버여야 합니다.
# 예: from langchain.vectorstores import FAISS
#      retriever = vectorstore.as_retriever()
# 각 쿼리에 대해 retriever를 호출하는 체인 구성:

# embedding_model = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
#     model_kwargs={'device': 'cpu'},  # 'cuda' 대신 'cpu' 사용
#     encode_kwargs={'normalize_embeddings': True}
# )


# FAISS 벡터스토어 생성 (전체 문서를 한 번에 인덱싱)
vectorstore = FAISS.from_documents(documents, embedding_model)

# 예시: vectorstore가 이미 생성되어 있다고 가정
retriever = vectorstore.as_retriever()

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

# --- 3. 최종 답변 생성 ---
# 최종 답변 프롬프트: 컨텍스트(융합된 문서)와 질문을 활용해 답변 생성
final_template = (
    "Answer the following question based on this context:\n\n"
    "{context}\n\n"
    "Question: {question}"
)
final_prompt = ChatPromptTemplate.from_template(final_template)

# 최종 답변에 사용할 LLM 인스턴스 (여기서는 ChatOpenAI 사용)
llm = ChatOpenAI(temperature=0)

# 융합된 문서들에서 컨텍스트를 추출하는 함수: 각 문서의 page_content를 이어 붙임
def get_context_from_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

# 최종 RAG 체인 구성: 컨텍스트와 질문을 받아 최종 답변 생성
final_rag_chain = (
    {
        "context": lambda inputs: get_context_from_docs(docs),  # retrieval_chain의 결과 사용
        "question": itemgetter("question")
    }
    | final_prompt
    | llm
    | StrOutputParser()
)

# --- 실행 예시 ---
# 질문 정의
question = "부산광역시 해운대구에 사는 28세 여자에게 맞는 서비스를 찾아줘. 생활안전에 도움이 될 현금 정책"

# 1단계: retrieval_chain을 통해 융합된 문서(컨텍스트) 얻기
docs = retrieval_chain_rag_fusion.invoke({"question": question})
print(f"Retrieved {len(docs)} fused documents.")

# 2단계: 최종 체인을 통해 답변 생성
final_answer = final_rag_chain.invoke({"question": question})
print("Final Answer:", final_answer)


NameError: name 'FAISS' is not defined

# rerank

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere


llm = ChatCohere()

system_prompt = (
    "Use the given context to answer the question in Korean. "
    "If you don't know the answer, say you don't know. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(compression_retriever, combine_docs_chain)

query = "서울에 1인가구 저소득층에 맞는 서비스를 찾아줘."
response = chain.invoke({"input": query})

print(response['answer']) 